In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
train_set = pd.read_csv('datasets/train.csv',header = 0)
print(train_set.shape)
print(train_set.columns)

In [ ]:
def dataPreprocess(dataset,test_check):
    dataset = dataset.drop(['id','hour'],axis=1)
    le = preprocessing.LabelEncoder()
    for elt in ['site_id','site_domain','site_category','app_id','app_domain','app_category','device_id','device_ip','device_model']:
        le.fit(dataset[elt])
        dataset[elt] = le.transform(dataset[elt])
    if test_check == 0:
        y = dataset['click']
        x = dataset.drop(['click'],axis = 1)
        return x,y
    else:
        return dataset

In [ ]:
train_sets = pd.read_csv('datasets/train.csv',chunksize = 100000,header = 0)

In [ ]:
count = 0
for train_set in train_sets:
    count += 1
print(count)

In [ ]:
test_set = pd.read_csv('datasets/test.csv',header = 0)

In [ ]:
clf = LogisticRegression(random_state = 52)
i = 0
for train_set in train_sets:
    if i <=10:
        x_train,y_train = dataPreprocess(train_set,0)
        clf.fit(x_train,y_train)
        i += 1
    elif i <=12:
        x_test,y_test = dataPreprocess(train_set,0)
        y_pred = clf.predict_proba(x_test)
        print(metrics.log_loss(y_test,y_pred))
        i += 1

In [ ]:
x_test = dataPreprocess(test_set,1)

In [ ]:
y_pred = clf.predict_proba(x_test)

In [ ]:
y_pred

## New Part

In [1]:
from csv import DictReader
from math import exp,log,sqrt

In [19]:
'''
function readData is to read the file using csv DictReader
the D is dimension, which is the range of the weight list
'''
def readData(path,D):
    for row in DictReader(open(path)):
        ID = row['id']
        del row['id']
        
        y = 0.
        if 'click' in row:
            if row['click'] == '1':
                y = 1.
            del row['click']
            
        row['hour'] = row['hour'][6:]
    
        x = []
        for key in row:
            value = row[key]
            #one-hot encode with hash trick
            index = abs(hash(key + '_'+value))%D
            x.append(index)
            
        yield ID,x,y

In [3]:
class logistic_regression(object):
    def __init__(self,alpha,L2,D,method):
        self.alpha = alpha
        self.L2 = L2
        
        self.D = D
        self.method = method
        
        self.w = [0.5]*D
        
    def _indices(self, x):
        method = self.method
        yield 0
        
        for index in x:
            yield index
            
        if method == 'interaction':
            D = self.D
            L = len(x)
            x = sorted(x)
            for i in range(1,L):
                for j in range(i+1, L):
                    # one-hot encode interactions with hash trick
                    yield abs(hash(str(x[i]) + '_' + str(x[j]))) % D
    def predict(self,x):
        w = self.w
        
        # h is the h(wx) in sigmoid function
        h = 0
        
        for i in self._indices(x):
            h += w[i]
            
        return 1./(1.+exp(-h))
    
    def update(self,x,p,y):
        # p is the prediction proba and y is the label
        g = p-y
        
        alpha = self.alpha
        L2 = self.L2
        w = self.w
        
        for i in self._indices(x):
            w[i] -= alpha*(g+ L2*w[i])

In [4]:
def log_loss(p, y):
    ''' FUNCTION: Bounded logloss

        INPUT:
            p: our prediction
            y: real answer

        OUTPUT:
            logarithmic loss of p given y
    '''

    p = max(min(p, 1. - 10e-15), 10e-15)
    return -log(p) if y == 1. else -log(1. - p)

In [20]:
log_learner = logistic_regression(0.01,0.**5,2**20,method = 'None')
count = 1
loss = 0
for ID,x,y in readData('datasets/train.csv',2**20):
    p = log_learner.predict(x)
    
    if count > 70000:
        loss += log_loss(p,y)
    else:
        log_learner.update(x,p,y)
    
    count += 1
    
    if count == 100001:
        print(loss/30000)
        break

0.4162688258379365


In [ ]:
metrics.log_loss([0],[1])

In [22]:
with open('datasets/submission_lr.csv', 'w') as outfile:
    outfile.write('id,click\n')
    for ID, x, y in readData('datasets/test.csv', 2**20):
        p = log_learner.predict(x)
        outfile.write('%s,%s\n' % (ID, str(p)))

PermissionError: [Errno 13] Permission denied

In [11]:
a = [(1,'a')]

In [12]:
a[0]

(1, 'a')

In [13]:
a[0][0]

1